In [1]:
import pandas as pd
import scipy as sc
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 10)

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")
etalon = pd.read_csv("etalon.csv")
#train = train.sample(frac=0.1, random_state=42)

In [3]:
all_data = pd.concat([train, test])

In [4]:
ohe = OneHotEncoder(sparse=False)

dummy_features = ohe.fit_transform(all_data['item_id'].values.reshape(-1, 1))
dummy_features = pd.DataFrame(dummy_features, columns=['good = ' + str(x) for x in range(dummy_features.shape[1])])
all_data.index = dummy_features.index
new_all_data = pd.concat([all_data, dummy_features], axis=1)

In [5]:
dummy_features = ohe.fit_transform(new_all_data['week'].values.reshape(-1, 1))
dummy_features = pd.DataFrame(dummy_features, columns=['week = ' + str(x) for x in range(dummy_features.shape[1])])
new_all_data.index = dummy_features.index
new_all_data = pd.concat([new_all_data, dummy_features], axis=1).drop(['week'], axis=1)

In [6]:
dummy_features = ohe.fit_transform(new_all_data['year'].values.reshape(-1, 1))
dummy_features = pd.DataFrame(dummy_features, columns=['year = ' + str(x) for x in range(dummy_features.shape[1])])
new_all_data.index = dummy_features.index
new_all_data = pd.concat([new_all_data, dummy_features], axis=1).drop(['year'], axis=1)

In [7]:
new_train = new_all_data.drop(['item_id', 'Num'], axis=1)[:len(train)]
new_test = new_all_data.drop(['item_id', 'Num'], axis=1)[len(train):]
new_test = new_test.drop(['y'], axis=1)

In [8]:
X_train, y_train = new_train.drop(['y'], axis=1).as_matrix(), new_train['y'].values
X_test = new_test.as_matrix()

In [9]:
class sum_regressor:
    def __init__(self):
        self.estimators = [RandomForestRegressor(n_estimators=17, min_samples_split=3), 
                          GradientBoostingRegressor(n_estimators=20, loss='lad'), 
                          ExtraTreesRegressor(n_estimators=20)]
    def fit(self, X, y):
        for est in self.estimators:
            est.fit(X, np.log(y))
            if est != self.estimators[-1]:
                np.c_[X, est.predict(X)]

    def predict(self, X):
        for est in self.estimators:
            if est != self.estimators[-1]:
                np.c_[X, est.predict(X)]
        return np.exp(self.estimators[-1].predict(X))

In [10]:
est = RandomForestRegressor(n_estimators=17, min_samples_split=3)
est.fit(X_train, y_train)
preds = est.predict(X_test)

In [11]:
sample_submission['y'] = preds
sample_submission.set_index('Num', inplace=True)
sample_submission.to_csv('submission.csv')

In [13]:
def test_estimator_onm(estimator):
    def do_stuff(X, y):
        tscv = TimeSeriesSplit(n_splits=10)
        score = []
        for train_index, test_index in tscv.split(X):
            estimator.fit(X[train_index], y[train_index])
            prediction = estimator.predict(X[test_index])
            score.append(SMAPE(y[test_index], prediction))
        print score
        return np.mean(score)
    return do_stuff(new_train.drop(['y'], axis=1).as_matrix(), new_train['y'].values)

def test_on_etalon(estimator):
    estimator.fit(X_train, y_train)
    return SMAPE(estimator.predict(X_test), etalon['y'].values)

def SMAPE(target, prediction):
    return 200*np.mean(np.abs(target-prediction)/(np.abs(target) + np.abs(prediction)))

24.8498838241


In [15]:
f60_train = train['f60']
f60_test = test['f60']

In [16]:
test['y'] = 0
test = test.set_index(np.linspace(1, test.shape[0], test.shape[0]).astype(int))
test['index1'] = test.index

In [17]:
for index in test['index1']:
    week = test.loc[(test['index1']==index)]['week'].values[0]
    item = test.loc[(test['index1']==index)]['item_id'].values[0]
    A = test.loc[(test['item_id'] == item) & (test['week']-test['shift']==week)]
    if A.empty == False:
        i = A.index[0]
        test.set_value(index, 'y', A.loc[test['index1']==i]['f60'].values[0]*1.61)
test = test.drop(['index1'], axis=1)

In [25]:
def preds_score(preds):
    index = test['y'] != 0
    print SMAPE(preds[index], etalon['y'].values[index])

In [26]:
preds_score(preds)

36.4668510141


/home/pavel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  app.launch_new_instance()
